# 事前準備

## matplotlib日本語化

In [3]:
!pip install japanize-matplotlib

                                              0.0/4.1 MB ? eta -:--:--
     --                                       0.3/4.1 MB 7.9 MB/s eta 0:00:01
     -------                                  0.8/4.1 MB 8.5 MB/s eta 0:00:01
     ------------                             1.3/4.1 MB 8.9 MB/s eta 0:00:01
     -----------------                        1.8/4.1 MB 10.3 MB/s eta 0:00:01
     ---------------------                    2.2/4.1 MB 10.1 MB/s eta 0:00:01
     --------------------------               2.8/4.1 MB 9.8 MB/s eta 0:00:01
     -------------------------------          3.2/4.1 MB 10.2 MB/s eta 0:00:01
     ------------------------------------     3.7/4.1 MB 10.4 MB/s eta 0:00:01
     ---------------------------------------  4.1/4.1 MB 10.1 MB/s eta 0:00:01
     ---------------------------------------- 4.1/4.1 MB 9.4 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: sta

## ライブラリのインポート

In [4]:
import numpy as np
import pandas as pd
import re
# データを表示する時に列を省略しないためのおまじない
pd.options.display.max_columns=None

from matplotlib import pyplot as plt
%matplotlib inline

# #日本語化matplotlibのインポート & 日本語のフォントを指定
import japanize_matplotlib
plt.rcParams['font.family'] = 'IPAexGothic'

from sklearn.linear_model import LinearRegression as LR
from sklearn.linear_model import Ridge
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error as MSE

# データの読み込み

In [5]:
train = pd.read_csv("./input/train_all.csv")
test = pd.read_csv("./input/test_all.csv")

# 投稿用のサンプルファイルの読み込み
sample = pd.read_csv("./input/sample_submit.csv",header=None)

In [6]:
train.head(2)

,id,y,year,stage,match,gameday,time,home,away,stadium,tv,home_score,away_score,weather,temperature,humidity,referee,home_team,home_01,home_02,home_03,home_04,home_05,home_06,home_07,home_08,home_09,home_10,home_11,away_team,away_01,away_02,away_03,away_04,away_05,away_06,away_07,away_08,away_09,away_10,away_11,address,capa
0,13994,18250,2012,Ｊ１,第１節第１日,03/10(土),14:04,ベガルタ仙台,鹿島アントラーズ,ユアテックスタジアム仙台,スカパー／ｅ２／スカパー光／ＮＨＫ総合,1,0,雨,3.8,66%,木村 博之,ベガルタ仙台,林 卓人,菅井 直樹,鎌田 次郎,上本 大海,田村 直也,富田 晋伍,角田 誠,太田 吉彰,関口 訓充,ウイルソン,赤嶺 真吾,鹿島アントラーズ,曽ヶ端 準,新井場 徹,岩政 大樹,中田 浩二,アレックス,青木 剛,増田 誓志,小笠原 満男,本山 雅志,大迫 勇也,ジュニーニョ,宮城県仙台市泉区七北田字柳78,19694
1,13995,24316,2012,Ｊ１,第１節第１日,03/10(土),14:04,名古屋グランパス,清水エスパルス,豊田スタジアム,スカパー／ｅ２／スカパー光（Ｊ ＳＰＯＲＴＳ ４）／ＮＨＫ名古屋,1,0,屋内,12.4,43%,西村 雄一,名古屋グランパス,楢﨑 正剛,田中 隼磨,田中 マルクス闘莉王,増川 隆洋,阿部 翔平,中村 直志,ダニルソン,藤本 淳吾,金崎 夢生,ケネディ,玉田 圭司,清水エスパルス,林 彰洋,吉田 豊,岩下 敬輔,カルフィン ヨン ア ピン,李 記帝,村松 大輔,河井 陽介,枝村 匠馬,高木 俊幸,アレックス,大前 元紀,愛知県豊田市千石町7-2,40000


# 前処理

訓練データとテストデータをけつごうしたいが、テストデータにyが無いから先に取り出しておく

In [7]:
y_train = train["y"]
train = train.drop("y",axis=1)

## 結合

In [8]:
# 結合前にtrainというフラグを作り、trainデータは1、testデータは0とする
train["train"]=1
test["train"]=0

# 結合
datas = pd.concat([train, test],sort=False)

## 不要なカラムの削除

In [9]:
drop_list = [
    "id",
    "tv",
    "referee",
    "time",
    "home",
    "away",
    "home_01", "home_02", "home_03", "home_04", "home_05", "home_06", "home_07", "home_08", "home_09", "home_10", "home_11",
    "away_01", "away_02", "away_03", "away_04", "away_05", "away_06", "away_07", "away_08", "away_09", "away_10", "away_11",
    "address",
]
datas = datas.drop(drop_list, axis=1)

# # datas.columns
datas.head(5)

,year,stage,match,gameday,stadium,home_score,away_score,weather,temperature,humidity,home_team,away_team,capa,train
0,2012,Ｊ１,第１節第１日,03/10(土),ユアテックスタジアム仙台,1,0,雨,3.8,66%,ベガルタ仙台,鹿島アントラーズ,19694,1
1,2012,Ｊ１,第１節第１日,03/10(土),豊田スタジアム,1,0,屋内,12.4,43%,名古屋グランパス,清水エスパルス,40000,1
2,2012,Ｊ１,第１節第１日,03/10(土),万博記念競技場,2,3,晴一時雨,11.3,41%,ガンバ大阪,ヴィッセル神戸,21000,1
3,2012,Ｊ１,第１節第１日,03/10(土),エディオンスタジアム広島,1,0,曇一時雨のち晴,11.4,52%,サンフレッチェ広島,浦和レッズ,50000,1
4,2012,Ｊ１,第１節第１日,03/10(土),札幌ドーム,0,0,屋内,22.5,32%,コンサドーレ札幌,ジュビロ磐田,39232,1


## 日付関係

In [10]:
datas["MONTH"] = datas["gameday"].apply(lambda x : x[0:2])
datas["DAY"] = datas["gameday"].apply(lambda x : x[3:5])
datas["WEEK"] = datas["gameday"].apply(lambda x : x[x.find("(")+1:x.find("(")+2])
datas = pd.get_dummies(datas, columns=["WEEK"], drop_first=True)
datas = datas.drop("gameday", axis=1)

## 試合？関係

In [11]:
datas["stage"] = datas["stage"].apply(lambda x : x[1])
datas["match_num"] = datas["match"].apply(lambda x : int(re.findall(r'\d+', x)[0]))
datas["match_day"] = datas["match"].apply(lambda x : int(re.findall(r'\d+', x)[1]))
datas = datas.drop("match", axis=1)

## チーム関係

In [12]:
datas = pd.get_dummies(datas, columns=["home_team"], drop_first=True)
datas = pd.get_dummies(datas, columns=["away_team"], drop_first=True)

## 説明変数および目的変数の抽出

- 学習データ（train）データからは説明変数と目的変数を、評価データ（test）からは説明変数のみを取り出します
- サンプルコードでは次のように変数の名前を付けます<br>
    - 学習データの説明変数： trainX
    - 学習データの目的変数： y
    - 評価データの説明変数： testX

In [13]:
# 説明変数を設定
drop_list = [
  "stadium",
  "weather",
	"humidity",
]

In [14]:
datas = datas.drop(drop_list, axis=1)

In [15]:
# datasにおいてtrainフラグが1のものをX_trainとする。その後、trainフラグを削除
X_train = datas[datas["train"]==1]
X_train = X_train.drop(columns="train")

# datasにおいてtrainフラグが0のものをX_testとする。その後、trainフラグを削除
X_test = datas[datas["train"]==0]
X_test = X_test.drop(columns="train")


# モデルごと


## LinearRegression

### 学習と予測

In [16]:
LRmodel = LR()
LRmodel.fit(X_train, y_train)
pred_LR = LRmodel.predict(X_test)

### 予測値の保存

In [17]:
sample[1] = pred_LR
sample.to_csv("LR.csv", index=None, header=None)

## LinearRegression

### 学習と予測

In [18]:
LRmodel = LR()
LRmodel.fit(X_train, y_train)
pred_LR = LRmodel.predict(X_test)

### 予測値の保存

In [19]:
sample[1] = pred_LR
sample.to_csv("LR.csv", index=None, header=None)

## Ridge

In [20]:
ridge = Ridge()
ridge.fit(X_train, y_train)
pred_ridge = ridge.predict(X_test)
sample[1] = pred_ridge
sample.to_csv("Ridge.csv", index=None, header=None)

ridge10 = Ridge(alpha=10)
ridge10.fit(X_train, y_train)
pred_ridge10 = ridge10.predict(X_test)
sample[1] = pred_ridge10
sample.to_csv("Ridge10.csv", index=None, header=None)

ridge01 = Ridge(alpha=0.1)
ridge01.fit(X_train, y_train)
pred_ridge01 = ridge01.predict(X_test)
sample[1] = pred_ridge01
sample.to_csv("Ridge01.csv", index=None, header=None)


## SVR

In [21]:
regr = SVR()
regr.fit(X_train, y_train)
pred_regr = regr.predict(X_test)
sample[1] = pred_regr
sample.to_csv("regr.csv", index=None, header=None)